In [1]:
import numpy as np
import torch
from torch.utils.data import Dataset
import pandas as pd
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import json
import time
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import init
import csv

In [2]:
np.random.seed(12345)

# run on the GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

class DataReader:
    NEGATIVE_TABLE_SIZE = 1e8

    def __init__(self, inputFileName, min_count):

        self.negatives = []
        self.discards = []
        self.negpos = 0

        self.word2id = dict()
        self.id2word = dict()
        self.sentences_count = 0
        self.token_count = 0
        self.word_frequency = dict()

        self.inputFileName = inputFileName
        self.read_words(min_count)
        self.initTableNegatives()
        self.initTableDiscards()
        
    def plot_frequency(self):
        pass
      
    def read_words(self, min_count):
        word_frequency = dict()
        for line in open(self.inputFileName, encoding="utf8"):
            line = line.split()
            if len(line) > 1:
                self.sentences_count += 1
                for word in line:
                    if len(word) > 0:
                        self.token_count += 1
                        word_frequency[word] = word_frequency.get(word, 0) + 1

                        if self.token_count % 1000000 == 0:
                            print("Read " + str(int(self.token_count / 1000000)) + "M words.")
        # show each word's frequency before the discard action
#         plot_frequency()
        wid = 0
        print()
        # w represents the word; c is the frequency of the word
        for w, c in word_frequency.items():
            if c < min_count:
                continue
            # if the counts of one word is less than min_count, then don't put this word in the vocabulary
            self.word2id[w] = wid
            self.id2word[wid] = w
            self.word_frequency[wid] = c
            wid += 1
        print("Total embeddings: " + str(len(self.word2id)))

    def initTableDiscards(self):
        t = 0.00001
        f = np.array(list(self.word_frequency.values())) / self.token_count
        # every ingredient's Probability to be discarded
        self.discards = np.sqrt(t / f) + (t / f)

    def initTableNegatives(self):
        pow_frequency = np.array(list(self.word_frequency.values())) ** 0.5
        words_pow = sum(pow_frequency)
        ratio = pow_frequency / words_pow
        count = np.round(ratio * DataReader.NEGATIVE_TABLE_SIZE)
        for wid, c in enumerate(count):
            self.negatives += [wid] * int(c)
        self.negatives = np.array(self.negatives)
        np.random.shuffle(self.negatives)

    def getNegatives(self, target, size):  # TODO check equality with target
        response = self.negatives[self.negpos:self.negpos + size]
        self.negpos = (self.negpos + size) % len(self.negatives)
        if len(response) != size:
            return np.concatenate((response, self.negatives[0:self.negpos]))
        return response

In [3]:
# -----------------------------------------------------------------------------------------------------------------

class Word2vecDataset(Dataset):
  # data is the object of class DataReader
    def __init__(self, data, window_size):
        self.data = data
        self.window_size = window_size
        self.input_file = open(data.inputFileName, encoding="utf8")

    def __len__(self):
        return self.data.sentences_count

    def __getitem__(self, idx):
        while True:
            line = self.input_file.readline()
            if not line:
                self.input_file.seek(0, 0)
                line = self.input_file.readline()

            if len(line) > 1:
                words = line.split()

                if len(words) > 1:
                    word_ids = [self.data.word2id[w] for w in words if
                                # according to the discard probabilty to decide keep this word or not so called: subsampling
                                w in self.data.word2id and np.random.rand() < self.data.discards[self.data.word2id[w]]]

                    boundary = np.random.randint(1, self.window_size)
                    # negative sampling
                    return [(u, v, self.data.getNegatives(v, 5)) for i, u in enumerate(word_ids) for j, v in
                            enumerate(word_ids[max(i - boundary, 0):i + boundary]) if u != v]

    @staticmethod
    def collate(batches):
        # u - center word
        all_u = [u for batch in batches for u, _, _ in batch if len(batch) > 0]
        # v - neighbor words
        all_v = [v for batch in batches for _, v, _ in batch if len(batch) > 0]
        all_neg_v = [neg_v for batch in batches for _, _, neg_v in batch if len(batch) > 0]

        return torch.LongTensor(all_u), torch.LongTensor(all_v), torch.LongTensor(all_neg_v)

In [4]:

"""
    u_embedding: Embedding for center word.
    v_embedding: Embedding for neighbor words.
"""


class SkipGramModel(nn.Module):

    def __init__(self, emb_size, emb_dimension):
        super(SkipGramModel, self).__init__()
        self.emb_size = emb_size
        self.emb_dimension = emb_dimension
        self.u_embeddings = nn.Embedding(emb_size, emb_dimension, sparse=True)
        self.v_embeddings = nn.Embedding(emb_size, emb_dimension, sparse=True)

        initrange = 1.0 / self.emb_dimension
        init.uniform_(self.u_embeddings.weight.data, -initrange, initrange)
        init.constant_(self.v_embeddings.weight.data, 0)

    def forward(self, pos_u, pos_v, neg_v):
        emb_u = self.u_embeddings(pos_u)
        emb_v = self.v_embeddings(pos_v)
        emb_neg_v = self.v_embeddings(neg_v)

        score = torch.sum(torch.mul(emb_u, emb_v), dim=1)
        score = torch.clamp(score, max=10, min=-10)
        score = -F.logsigmoid(score)

        neg_score = torch.bmm(emb_neg_v, emb_u.unsqueeze(2)).squeeze()
        neg_score = torch.clamp(neg_score, max=10, min=-10)
        neg_score = -torch.sum(F.logsigmoid(-neg_score), dim=1)

        return torch.mean(score + neg_score)

    def save_embedding(self, id2word, file_name):
        embedding = self.u_embeddings.weight.cpu().data.numpy()
        with open(file_name, 'w') as f:
            csv_writer = csv.writer(f)
            csv_writer.writerow(['Ingredient', 'Vector'])
            for wid, w in id2word.items():
                vector = str(list(embedding[wid]))
                csv_writer.writerow([w,vector])
        return embedding, id2word

In [5]:
import torch
import torch.optim as optim
from torch.utils.data import DataLoader
from tqdm import tqdm

# from data_reader import DataReader, Word2vecDataset
# from model import SkipGramModel


class Word2VecTrainer:
    def __init__(self, input_file, output_file, emb_dimension=100, batch_size=32, window_size=5, iterations=10,
                 initial_lr=0.001, min_count=5):

        self.data = DataReader(input_file, min_count)
        # dataset is the object of class Word2vecDataset
        dataset = Word2vecDataset(self.data, window_size)
        self.dataloader = DataLoader(dataset, batch_size=batch_size,
                                     shuffle=False, num_workers=0, collate_fn=dataset.collate)

        self.output_file_name = output_file
        self.emb_size = len(self.data.word2id)
        self.emb_dimension = emb_dimension
        self.batch_size = batch_size
        self.iterations = iterations
        self.initial_lr = initial_lr
        # put model on the GPU
        self.skip_gram_model = SkipGramModel(self.emb_size, self.emb_dimension).to(device)


    def train(self):

        for iteration in range(self.iterations):

            print("\n\n\nIteration: " + str(iteration + 1))
            optimizer = optim.SparseAdam(self.skip_gram_model.parameters(), lr=self.initial_lr)
            scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, len(self.dataloader))

            running_loss = 0.0
            for i, sample_batched in enumerate(tqdm(self.dataloader)):

                if len(sample_batched[0]) > 1:
                    # put training data on the GPU
                    pos_u = sample_batched[0].to(device)
                    pos_v = sample_batched[1].to(device)
                    neg_v = sample_batched[2].to(device)
                    optimizer.step()
                    scheduler.step()
                    optimizer.zero_grad()
                    loss = self.skip_gram_model.forward(pos_u, pos_v, neg_v)
                    loss.backward()
                    running_loss = running_loss * 0.9 + loss.item() * 0.1
                    if i > 0 and i % 500 == 0:
                        print(" Loss: " + str(running_loss))
        self.skip_gram_model.save_embedding(self.data.id2word, self.output_file_name)
        return self.skip_gram_model 

In [6]:
inputFileName = 'ingredients_1127.txt'
w2v = Word2VecTrainer(input_file=inputFileName, output_file="food2vec_1127.csv")
skip_gram_model = w2v.train()

Read 1M words.

Total embeddings: 7032


  0%|          | 0/5571 [00:00<?, ?it/s]




Iteration: 1


  9%|▉         | 517/5571 [00:15<00:55, 91.42it/s]

 Loss: 4.139847924957127


...

 90%|████████▉ | 5008/5571 [00:57<00:08, 67.74it/s]

 Loss: 2.367199815999722


 99%|█████████▉| 5508/5571 [01:05<00:00, 79.97it/s]

 Loss: 2.3233798079621124


100%|██████████| 5571/5571 [01:06<00:00, 84.34it/s]


In [7]:
skip_gram_model

SkipGramModel(
  (u_embeddings): Embedding(7032, 100, sparse=True)
  (v_embeddings): Embedding(7032, 100, sparse=True)
)